# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [30]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import pickle

import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.datasets import make_multilabel_classification
from sklearn.metrics import classification_report


url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/charlesliu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/charlesliu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/charlesliu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [31]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql('SELECT * FROM disaster_respone', engine)
X = df['message']
y = df.iloc[:, 4:]

### 2. Write a tokenization function to process your text data

In [44]:
print(df.shape)
df.head()


(26216, 40)


,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [39]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [40]:
y_train.mean()

related                   0.775510
request                   0.170656
offer                     0.004387
aid_related               0.416222
medical_help              0.079249
medical_products          0.049733
search_and_rescue         0.028467
security                  0.018262
military                  0.033616
child_alone               0.000000
water                     0.063561
food                      0.111005
shelter                   0.087927
clothing                  0.015831
money                     0.023841
missing_people            0.011348
refugees                  0.033569
death                     0.045632
other_aid                 0.131175
infrastructure_related    0.065659
transport                 0.045823
buildings                 0.050830
electricity               0.019550
tools                     0.006056
hospitals                 0.011301
shops                     0.004673
aid_centers               0.011539
other_infrastructure      0.044249
weather_related     

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [41]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7fbe95e2acb0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

In [42]:
y_pred = pipeline.predict(X_test)
y_pred

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [54]:
with open ('classifier.pkl', 'wb') as f:
    pickle.dump(pipeline, f)

In [52]:
# for i in range(len(y_test.columns)):
#     # print(y_test.iloc[:, i], y_pred[:, i])
#     # print(type(classification_report(y_test.iloc[:, i], y_pred[:, i])))
#     print(y_test.columns[i])
#     print(classification_report(y_test.iloc[:, i], y_pred[:, i], output_dict=True)['weighted avg'])
    

for i, category in enumerate(y_test.columns):
    print(category)
    # print(y_test.iloc[:, i])
    # print(y_pred[: i])
    print(classification_report(y_test[category], y_pred[:, i]))

related
              precision    recall  f1-score   support

           0       0.75      0.25      0.38      1266
           1       0.80      0.97      0.88      3938
           2       0.80      0.10      0.18        40

    accuracy                           0.79      5244
   macro avg       0.78      0.44      0.48      5244
weighted avg       0.78      0.79      0.75      5244

request
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      4349
           1       0.89      0.42      0.57       895

    accuracy                           0.89      5244
   macro avg       0.89      0.70      0.75      5244
weighted avg       0.89      0.89      0.87      5244

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

    accuracy                           1.00      5244
   macro avg       0.50      0.50      0.50      524

/Users/charlesliu/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/charlesliu/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/charlesliu/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.93      0.99      0.96      4649
           1       0.88      0.38      0.54       595

    accuracy                           0.92      5244
   macro avg       0.90      0.69      0.75      5244
weighted avg       0.92      0.92      0.91      5244

shelter
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      4774
           1       0.84      0.20      0.32       470

    accuracy                           0.92      5244
   macro avg       0.88      0.60      0.64      5244
weighted avg       0.92      0.92      0.90      5244

clothing
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5171
           1       1.00      0.03      0.05        73

    accuracy                           0.99      5244
   macro avg       0.99      0.51      0.52      5244
weighted avg       0.99      0.99      0.98      5244

mo

/Users/charlesliu/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/charlesliu/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/charlesliu/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Use

### 6. Improve your model
Use grid search to find better parameters. 

In [37]:
parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__max_features': (None, 5000, 10000),
    'tfidf__use_idf': (True, False),
    'clf__estimator__n_estimators': [50, 100, 200],
    'clf__estimator__min_samples_split': [2, 3, 4],
}

parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),
}


cv = GridSearchCV(pipeline, param_grid=parameters)

cv.fit(X_train, y_train)

model = cv.best_estimator_

y_pred = model.predict(X_test)

print("\nBest Parameters:", cv.best_params_)

KeyboardInterrupt: 

In [ ]:
for i in range(len(y_test.columns)):
    # print(y_test.iloc[:, i], y_pred[:, i])
    # print(type(classification_report(y_test.iloc[:, i], y_pred[:, i])))
    print(y_test.columns[i])
    print(classification_report(y_test.iloc[:, i], y_pred[:, i], output_dict=True)['weighted avg'])

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.